In [3]:
%pylab inline
plt.style.use("default")
import os, sys
import scipy
import pandas as pd
import helpers as h
import utils as u
from glob import glob
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
from sklearn.cluster import KMeans
from mpl_toolkits import mplot3d
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

plt.rcParams['figure.figsize'] = (15, 10) 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
data_root = "/Users/duuta/ppp/data/stringer/live_data/"
data_files = [fname.split("/")[-1] for fname in glob(f"{data_root}natimg2800_M*.mat")]



In [5]:
def clean_unpack(data0):
    """returns clean and zscored signal"""
    resp0, spon0, istim0 = h.unbox(data0)

    resp0 = h.denoise_resp(resp0, spon0)

    resp0 = h.dupSignal(resp0, istim0)

    return resp0


In [6]:
data = loadmat( os.path.join(data_root, data_files[0]), simplify_cells=True)
resp = clean_unpack(data)



In [7]:
x, y, z = data['med'].T


In [8]:
def points_above_line(a, b, c, point_list):
    above_indices, below_indices = [], []
    
    for i, point in enumerate(zip(point_list[0], point_list[1])):
        x, y = point
        if a*x + b*y + c > 0:
            above_indices.append(i)
        else:
            below_indices.append(i)


    return above_indices, below_indices



In [ ]:
point_list = [x, y]


In [44]:
a, b, c = 59/70, 1, -1168.571

In [52]:
above_points, below_points = points_above_line(a, b, c, point_list)

In [53]:
nab = resp[:, :, above_points]
nbl = resp[:, :, below_points]


In [54]:
def compute_cvPCA(resp0):
    ss0 = u.shuff_cvPCA(resp0, nshuff=10)
    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0


In [55]:
ss0 = compute_cvPCA(nab)

In [56]:
ss1 = compute_cvPCA(nbl)

In [57]:
aalpha, _ = u.get_powerlaw(ss0,np.arange(11, 200).astype('int') )
balpha, _ = u.get_powerlaw(ss1, np.arange(11, 200).astype('int'))


In [50]:
aalpha

1.081978663083578

In [58]:
balpha

1.022544387723973